## Setup

In [ ]:
%%capture
!uv pip install --system --no-index --find-links=/kaggle/input/rsna2025-packages/pip-packages -r /kaggle/input/rsna2025-pip-packages/requirements.txt --no-build-isolation 
!uv pip install --system --no-index --find-links=/kaggle/input/rsna2025-packages/pip-packages --force-reinstall numpy==1.26.4

In [ ]:
%%capture
!cp -r /kaggle/input/rsna2025-nnunet /kaggle/working/
!rm -r /kaggle/working/rsna2025-nnunet/nnunetv2.egg-info;
!cd /kaggle/working/rsna2025-nnunet; uv pip install --system . --no-build-isolation --no-index

In [ ]:
%%capture
!dpkg -i /kaggle/input/rsna2025-packages/libyaml-cpp0.7_0.7.0+dfsg-8build1_amd64.deb
!dpkg -i /kaggle/input/rsna2025-packages/dcm2niix_1.0.20211006-1build1_amd64.deb
!dpkg -i /kaggle/input/rsna2025-packages/libgdcm-tools_3.0.10-1build2_amd64.deb

In [ ]:
!cp -r /kaggle/input/rsna2025-src /kaggle/working; mv /kaggle/working/rsna2025-src /kaggle/working/src
!cp -r /kaggle/input/rsna2025-scripts /kaggle/working; mv /kaggle/working/rsna2025-scripts /kaggle/working/scripts
!ln -s /kaggle/input/rsna2025-configs /kaggle/working/configs

!mkdir -p /kaggle/working/logs/train/runs
!ln -s "/kaggle/input/251013-prev6-1-ex-dav6w3-e25-w01-005-1-s128-256/251013-seg_tf-v4-nnunet_truncate1_preV6_1-ex_dav6w3-m32g64-e25-w01_005_1-s128_256_256" /kaggle/working/logs/train/runs/

# for rootutils
!touch /kaggle/working/.project-root

## Inference

In [ ]:
import rootutils
import os
from pathlib import Path

rootutils.setup_root(Path().resolve(), indicator=".project-root", pythonpath=True)

root = os.environ["PROJECT_ROOT"]
os.chdir(root)

from scripts.rsna_submission_roi import predict

In [ ]:
%env VESSEL_DEVICE=cuda:0
%env VESSEL_DEVICES=0

# Whether to abort when sparse search fails
%env VESSEL_ABORT_ON_SPARSE_FAIL=1
%env VESSEL_ABORT_MIN_ALL_DIMS_MM=140
# Whether to abort when the ROI volume is too small
%env VESSEL_ABORT_ON_SMALL_ROI=1  
%env VESSEL_MIN_ROI_VOXELS=1000000
# Whether to abort when the number of union voxels is too small
%env VESSEL_ABORT_ON_LOW_UNION=1  
%env VESSEL_MIN_UNION_SUM=5000

%env RSNA_ERROR_FALLBACK_PROBS=0.02,0.02,0.08,0.08,0.03,0.03,0.07,0.02,0.02,0.02,0.02,0.02,0.02,0.35

# Vessel Segmentation
%env VESSEL_NNUNET_SPARSE_MODEL_DIR=/kaggle/input/nnunet-vessel-grouping-da7
%env VESSEL_ENABLE_ORIENTATION_CORRECTION=1
%env VESSEL_ORIENTATION_WEIGHTS=1,1,1
# ROI Size [mm]
%env VESSEL_SPARSE_ROI_EXTENT_MM=140
%env VESSEL_REFINE_Z_ONLY=0

# %env VESSEL_NNUNET_MODEL_DIR=/kaggle/input/nnunet-da3-sklr-with-all
%env VESSEL_NNUNET_MODEL_DIR=/kaggle/input/nnunet-da3-sklr-ep800
%env VESSEL_ADDITIONAL_DENSE_MODEL_DIRS=/kaggle/input/nnunet-da6-sklr-w3-tv07
%env VESSEL_FOLDS=all

%env VESSEL_REFINE_MARGIN_Z=15
%env VESSEL_REFINE_MARGIN_XY=30
%env VESSEL_SPARSE_OVERLAP=0.2
%env VESSEL_DENSE_OVERLAP=0.3

%env VESSEL_TRT_DIR=/kaggle/input/rsna2025-convert-trt/trt

# ROI Classification Model
%env ROI_EXPERIMENTS=251013-seg_tf-v4-nnunet_truncate1_preV6_1-ex_dav6w3-m32g64-e25-w01_005_1-s128_256_256
%env ROI_FOLDS=0,1,2,3
%env ROI_TTA=2
%env ROI_NNUNET_MODEL_DIR=/kaggle/input/nnunet-da6-1-sklr-tv07

%env CONFIG_DIR=/kaggle/working/configs
%env RUN_MODE=kaggle

%env RSNA_ONLY_AP=0
%env RSNA_ONLY_LOCATIONS=0

%env RSNA_MAX_PREDICT_ERRORS=300
# %env RSNA_STAGE=dicom
# %env RSNA_VESSEL_STAGE_LIMIT=roi_crop 
# "load"|"preprocess"|"sparse_search"|"roi_crop"|"fold_infer"|"refine"|"aggregate"

if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    %env RSNA_DEBUG=1

# warmup
for i in range(3):
    predict("/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647")

In [ ]:
from kaggle_evaluation.rsna_inference_server import RSNAInferenceServer
import polars as pl

inference_server = RSNAInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway()
    display(pl.read_parquet('submission.parquet'))
